### Generating Coco Annotations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nibabel -q
!pip install pybboxes -q
!pip install sahi -q
!pip install -i https://test.pypi.org/simple/ supervision==0.3.0
!pip install -q timm -q
!pip install pycocotools -q
!pip install opencv-python -q
!pip install -i https://test.pypi.org/simple/ supervision==0.3.0 -1

# # !pip install tensorboard -q

# # !pip install --upgrade torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.0 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -1


In [ ]:
import pathlib as plb
import tempfile
import os

import shutil
import pandas as pd


import nibabel as nib
import numpy as np

import sys
import shutil

import pandas as pd
from tqdm import tqdm
from PIL import Image

import enum
import time
import random
import multiprocessing
from pathlib import Path

import torch
import torchvision

import torch.nn.functional as F
import torchvision.transforms as T

import numpy as np
# from unet import UNet
from scipy import stats
import matplotlib.pyplot as plt

from IPython import display
from tqdm.auto import tqdm
from pathlib import Path
import pandas as pd
import os
from torchvision.utils import save_image
from skimage.measure import regionprops_table
from sklearn.model_selection import train_test_split

In [ ]:
### Note :if you have access to below folder you do NOT need to change them to your local directories, all file readings are read only
## Constants
BATCH_SIZE=8

## Checkpoint
CHECKPOINT_PATH = '/home/vk2501/checkpoints'

## Base Path
BASE_PATH = '/home/vk2501/data/DETR_ALL/SUV_DETR'

## DETR Path
# DETR = '/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/SPLIT_v2'
DETR = '/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans'

EVAL_BATCH_ITER=10

## Number of epochs
EPOCHS=10

## Dimensions of each image
DIM = 408

In [ ]:
DETR

'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans'

In [ ]:
# Reading the CSV files into lists
train_df = pd.read_csv(DETR + '/train.csv').rename(columns={'img_filename': 'slice'})
train_set = train_df['slice'].tolist()

validation_df = pd.read_csv(DETR + '/val.csv').rename(columns={'img_filename': 'slice'})
validation_set = validation_df['slice'].tolist()

test_df = pd.read_csv(DETR + '/test.csv').rename(columns={'img_filename': 'slice'})
test_set = test_df['slice'].tolist()

In [ ]:
print("Length of the split")
print("length : train",len(train_set))
print("length : val", len(validation_set))
print("length : test", len(test_set))

Length of the split
length : train 20083
length : val 8091
length : test 3984


In [ ]:
print("Length of the split")
print("length : train",len( set(train_set) ))
print("length : val", len( set(validation_set)))
print("length : test", len( set( test_set)))

Length of the split
length : train 14754
length : val 5913
length : test 2954


### Convert to COCO format

In [ ]:
import pybboxes as pbx
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation
from sahi.utils.file import save_json

In [ ]:
def coco_format(xmin,ymin,xmax,ymax, width, height):
  x = xmin / width
  y = ymin / height
  w = (xmax - xmin) / width
  h = (ymax - ymin) / height

  return x, y, w, h

### Annnotation Function

In [ ]:
category_id_map = {
        'lung_cancer' : 1,
        'lymphoma' : 2,
        'melanoma' : 3,
        'negative' :0
}


# not_consider = ['PETCT_d69c3fceba_PETCTstacked_axial_292.jpg', 'PETCT_36d8219e3f_PETCTstacked_axial_700.jpg']
## x1, y1, x2, y2 || < > Try with a check and
def convert2coco(samples,  negative_images,file_location, create_negative):

          coco = Coco()

          coco.add_category(CocoCategory(id=category_id_map['lung_cancer'], name='lung_cancer'))
          coco.add_category(CocoCategory(id=category_id_map['lymphoma'], name='lymphoma'))
          coco.add_category(CocoCategory(id=category_id_map['melanoma'], name='melanoma'))
          # coco.add_category(CocoCategory(id=category_id_map['negative'], name='negative'))

          for index, row in tqdm(samples.iterrows()):
                image= (row['slice'])

                # if image in not_consider:
                #   print(image)
                #   continue

                selected_row = row

                xmin = selected_row['x_min']
                xmax = selected_row['x_max']
                ymin = selected_row['y_min']
                ymax = selected_row['y_max']
                cancer_type = selected_row['cancer_type']
                voc_bbox = (xmin,ymin,xmax,ymax)


                xmin,ymin,width,height = pbx.convert_bbox(voc_bbox, from_type="voc", to_type="coco")
                # xmin,ymin,width,height = coco_format(xmin,ymin,xmax,ymax, DIM, DIM)

                coco_image = CocoImage(file_name=image, height=DIM, width=DIM)
                coco_image.add_annotation(
                  CocoAnnotation(
                  bbox=[xmin, ymin, width, height],
                  category_id=category_id_map[cancer_type],
                  category_name=cancer_type
                  )
                  )

                coco.add_image(coco_image)
          if(create_negative):
            print('creating_negative')
            cancer_type = 'negative'
            for negative_image in tqdm(negative_images):
                  image = str(negative_image)
                  if image in not_consider:
                    print(image)
                    continue
                  coco_image = CocoImage(file_name=image, height=DIM, width=DIM)
                  coco.add_image(coco_image)

          save_json(data=coco.json, save_path=file_location)


### K-Means Annotations

In [ ]:

category_id_map = {
        'lung_cancer' : 1,
        'lymphoma' : 2,
        'melanoma' : 3,
        'negative' :0
}

error_slices = 'name'
# not_consider = ['PETCT_d69c3fceba_PETCTstacked_axial_292.jpg', 'PETCT_36d8219e3f_PETCTstacked_axial_700.jpg']
## x1, y1, x2, y2 || < > Try with a check and
def convert2coco_k_means(samples,  negative_images,file_location, create_negative):
          coco = Coco()
          bounding_box_error_slices = []
          bounding_box_error_slices_number = []
          multiple_annotations_slices = []

          coco.add_category(CocoCategory(id=category_id_map['lung_cancer'], name='lung_cancer'))
          coco.add_category(CocoCategory(id=category_id_map['lymphoma'], name='lymphoma'))
          coco.add_category(CocoCategory(id=category_id_map['melanoma'], name='melanoma'))
          # coco.add_category(CocoCategory(id=category_id_map['negative'], name='negative'))

          for  slice_name, group_data in (samples):

              image= slice_name

                # if image in not_consider:
                #   print(image)
                #   continue

              coco_image = CocoImage(file_name=image, height=DIM, width=DIM)

              if len(group_data)> 1:
                    # print(f"Group: {slice_name}")
                    # print(len(group_data))
                    multiple_annotations_slices.append(image)
                    cur_index = -1
                    for index, selected_row in group_data.iterrows():
                              cur_index+=1
                              xmin = selected_row['x_min']
                              xmax = selected_row['x_max']
                              ymin = selected_row['y_min']
                              ymax = selected_row['y_max']
                              cancer_type = selected_row['cancer_type']
                              voc_bbox = (xmin,ymin,xmax,ymax)
                              # print(voc_bbox)
                              try:
                                xmin,ymin,width,height = pbx.convert_bbox(voc_bbox, from_type="voc", to_type="coco")
                              except Exception as e :

                                bounding_box_error_slices.append(image)
                                bounding_box_error_slices_number.append(cur_index)

                                continue

                              coco_image.add_annotation(
                                                CocoAnnotation(
                                                bbox=[xmin, ymin, width, height],
                                                category_id=category_id_map[cancer_type],
                                                category_name=cancer_type
                                                )
                                                )
              else:
                      selected_row =  group_data.iloc[0]
                      xmin = selected_row['x_min']
                      xmax = selected_row['x_max']
                      ymin = selected_row['y_min']
                      ymax = selected_row['y_max']
                      cancer_type = selected_row['cancer_type']
                      voc_bbox = (xmin,ymin,xmax,ymax)

                      xmin,ymin,width,height = pbx.convert_bbox(voc_bbox, from_type="voc", to_type="coco")
                      # xmin,ymin,width,height = coco_format(xmin,ymin,xmax,ymax, DIM, DIM)

                      coco_image.add_annotation(
                        CocoAnnotation(
                        bbox=[xmin, ymin, width, height],
                        category_id=category_id_map[cancer_type],
                        category_name=cancer_type
                        )
                        )
              ##  add the image
              coco.add_image(coco_image)

          if(create_negative):
            print('creating_negative')
            cancer_type = 'negative'
            for negative_image in tqdm(negative_images):
                  image = str(negative_image)

                  # if image in not_consider:
                  #   print(image)
                  #   continue
                  coco_image = CocoImage(file_name=image, height=DIM, width=DIM)
                  coco.add_image(coco_image)

          save_json(data=coco.json, save_path=file_location)

          df = pd.DataFrame({'slice_names': bounding_box_error_slices, 'slice_numbers': bounding_box_error_slices_number})

          df.to_csv(f'/content/sample_data/error_slices_{error_slices}.csv')

In [ ]:
labels_path = '/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/train_test_split_master_v2.csv'

columns =['file_name','cancer_type','split']

df = pd.read_csv(labels_path, usecols=columns ).rename(columns={'file_name': 'slice'})
df.head()

,slice,cancer_type,split
0,PETCT_2dac5ef654_axial_277.jpg,lung_cancer,train
1,PETCT_bf178a41b2_axial_234.jpg,lung_cancer,train
2,PETCT_37952b7ffb_axial_277.jpg,lung_cancer,train
3,PETCT_e03b96666f_axial_103.jpg,lung_cancer,train
4,PETCT_760c77b289_axial_259.jpg,lung_cancer,train


In [ ]:
columns =['img_filename','x_min','y_min','x_max','y_max','cancer_type','img_width','img_height']

In [ ]:
train = DETR+'/train.csv'
file_location = DETR + "/annotations/train/_annotations_kmeans_positive.coco.json"
print(train)

train= pd.read_csv(train, usecols=columns ).rename(columns={'img_filename': 'slice'}).groupby(by ='slice')
# train['slice'] = train['slice'].apply(lambda x: x.replace('_axial_', '_PETCTstacked_axial_'))
train.head()

negative = df[(df['split'] == 'train') & (df['cancer_type'] == 'negative')]
negative.head()
# negative =  list ( set ( [name.replace('_axial_', '_PETCTstacked_axial_') for name in negative['slice']]))
negative =  list ( set ( negative['slice'] ) )

print(len(train))
print(len(negative))
print(len(train) + len(negative))
error_slices = 'train_positive'
convert2coco_k_means(train, negative, file_location,create_negative=False)


train = DETR+'/train.csv'
file_location = DETR + "/annotations/train/_annotations_kmeans_all.coco.json"
print(train)

train= pd.read_csv(train, usecols=columns ).rename(columns={'img_filename': 'slice'}).groupby(by ='slice')
# train['slice'] = train['slice'].apply(lambda x: x.replace('_axial_', '_PETCTstacked_axial_'))
train.head()

negative = df[(df['split'] == 'train') & (df['cancer_type'] == 'negative')]
negative.head()
# negative =  list ( set ( [name.replace('_axial_', '_PETCTstacked_axial_') for name in negative['slice']]))
negative =  list ( set ( negative['slice'] ) )
print(len(train))
print(len(negative))
print(len(train) + len(negative))
error_slices = 'train_negative'
convert2coco_k_means(train, negative, file_location,create_negative=True)

/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans/train.csv
14754
5000
19754
/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans/train.csv
14754
5000
19754
creating_negative


  0%|          | 0/5000 [00:00<?, ?it/s]

### Errors and issues

In [ ]:
error_slices = pd.read_csv('/content/sample_data/error_slices.csv').drop(columns='Unnamed: 0')
error_slices.describe()

,slice_numbers
count,19.000000
mean,1.578947
std,1.346427
min,0.000000
25%,0.500000
50%,1.000000
75%,2.500000
max,4.000000


In [ ]:
train = DETR+'/train.csv'
file_location = DETR + "/annotations/train/_annotations_kmeans_positive.coco.json"
print(train)

train= pd.read_csv(train, usecols=columns ).rename(columns={'img_filename': 'slice'})
train[train['slice'] == 'PETCT_01140d52d8_axial_334.jpg']

/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans/train.csv


,slice,x_min,y_min,x_max,y_max,cancer_type,img_width,img_height
16021,PETCT_01140d52d8_axial_334.jpg,127.0,137.0,209.0,222.0,melanoma,408,408.0
16022,PETCT_01140d52d8_axial_334.jpg,136.0,148.0,211.0,223.0,melanoma,408,408.0
16023,PETCT_01140d52d8_axial_334.jpg,163.0,190.0,130.0,136.0,melanoma,408,408.0
16024,PETCT_01140d52d8_axial_334.jpg,146.0,155.0,130.0,137.0,melanoma,408,408.0
16025,PETCT_01140d52d8_axial_334.jpg,130.0,144.0,203.0,213.0,melanoma,408,408.0


In [ ]:
train = DETR+'/train.csv'
file_location = DETR + "/annotations/train/_annotations_kmeans_positive.coco.json"
print(train)

train= pd.read_csv(train, usecols=columns ).rename(columns={'img_filename': 'slice'})
train[train['slice'] == 'PETCT_01140d52d8_axial_334.jpg']

/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans/train.csv


,slice,x_min,y_min,x_max,y_max,cancer_type,img_width,img_height
16021,PETCT_01140d52d8_axial_334.jpg,127.0,137.0,209.0,222.0,melanoma,408,408.0
16022,PETCT_01140d52d8_axial_334.jpg,136.0,148.0,211.0,223.0,melanoma,408,408.0
16023,PETCT_01140d52d8_axial_334.jpg,163.0,190.0,130.0,136.0,melanoma,408,408.0
16024,PETCT_01140d52d8_axial_334.jpg,146.0,155.0,130.0,137.0,melanoma,408,408.0
16025,PETCT_01140d52d8_axial_334.jpg,130.0,144.0,203.0,213.0,melanoma,408,408.0


##  Test

In [ ]:
test = DETR+'/test.csv'
file_location = DETR +"/annotations/test/_annotations_kmeans_positive.coco.json"
print(test)
test= pd.read_csv(test, usecols=columns ).rename(columns={'img_filename': 'slice'}).groupby(by ='slice')
# test['slice'] = test['slice'].apply(lambda x: x.replace('_axial_', '_PETCTstacked_axial_'))
negative = df[(df['split'] == 'test') & (df['cancer_type'] == 'negative')]
negative =  list ( set ( negative['slice'] ) )
error_slices = 'test_positive'
convert2coco_k_means(test, negative, file_location,create_negative=False)


test = DETR+'/test.csv'
file_location = DETR +"/annotations/test/_annotations_kmeans_all.coco.json"

print(test)

test= pd.read_csv(test, usecols=columns ).rename(columns={'img_filename': 'slice'}).groupby(by ='slice')

# test['slice'] = test['slice'].apply(lambda x: x.replace('_axial_', '_PETCTstacked_axial_'))
# negative =  list ( set ( [name.replace('_axial_', '_PETCTstacked_axial_') for name in negative['slice']]))

negative = df[(df['split'] == 'test') & (df['cancer_type'] == 'negative')]
negative.head()

negative =  list ( set ( negative['slice'] ) )

error_slices = 'test_negative'
convert2coco_k_means(test, negative, file_location,create_negative=True)

/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans/test.csv
/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans/test.csv
creating_negative


  0%|          | 0/1000 [00:00<?, ?it/s]

#### Validation

In [ ]:
val = DETR+'/val.csv'
file_location =  DETR +"/annotations/val/_annotations_kmeans_positive.coco.json"
print(val)
val= pd.read_csv(val ,usecols=columns ).rename(columns={'img_filename': 'slice'}).groupby(by ='slice')
val.head()
negative = df[(df['split'] == 'val') & (df['cancer_type'] == 'negative')]
error_slices = 'val_positive'
convert2coco_k_means(val, negative, file_location,create_negative=False)


val = DETR+'/val.csv'
file_location =  DETR +"/annotations/val/_annotations_kmeans_all.coco.json"
print(val)
val= pd.read_csv(val ,usecols=columns ).rename(columns={'img_filename': 'slice'}).groupby(by ='slice')
val.head()
negative = df[(df['split'] == 'val') & (df['cancer_type'] == 'negative')]
negative =  list ( set ( negative['slice'] ) )
error_slices = 'val_negative'
convert2coco_k_means(val, negative, file_location,create_negative=True)

/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans/val.csv
/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/k_means/DETR/SUV_DETR_kmeans/val.csv
creating_negative


  0%|          | 0/2000 [00:00<?, ?it/s]